# How to Use:
This Notebook should ease the use of the API for "cleanup" tasks. It includes some sample for specific use cases that can be adjusted.
1. run first block to prepare excecutions by creating api connection
2. run any following block to execute a specific function

In [ ]:
import json
import logging
import logging.config
from pathlib import Path

from churchtools_api.churchtools_api import ChurchToolsApi
from secure.config import ct_domain, ct_token

logger = logging.getLogger(__name__)

config_file = Path("logging_config.json")
with config_file.open(encoding="utf-8") as f_in:
    logging_config = json.load(f_in)
    log_directory = Path(logging_config["handlers"]["file"]["filename"]).parent
    if not log_directory.exists():
        log_directory.mkdir(parents=True)
    logging.config.dictConfig(config=logging_config)

# Create Session


api = ChurchToolsApi(ct_domain)
api.login_ct_rest_api(ct_token=ct_token)

## Songs & Tag - mass change
Helper to iterate all songs

In [ ]:
songs = api.get_songs()
print('Got {} songs and the first one is "{}"'.format(len(songs), songs[0]["name"]))  # noqa: T201

Mass change append a tag to all songs - e.g. 51 is tag:"in ChurchTools vor Skript Import"

In [ ]:
all_song_ids = [value["id"] for value in songs]
for song_id in all_song_ids:
    api.add_song_tag(song_id=song_id, song_tag_id=51)

# Convert Song Category / Number to SongSource SongReference in Default Arrangement

In [ ]:
songs = api.get_songs()

In [ ]:
import time

CATEGORY_MAPPING = {1: 10, 3: 10, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 15: 9, 9: 11}
for iteration, song in enumerate(songs):
    if song["category"]["id"] in CATEGORY_MAPPING:
        arrangement_id = api.get_song_arrangement(song_id=song["id"])["id"]
        song_source = CATEGORY_MAPPING[song["category"]["id"]]
        song_reference = song["name"].split(" ")[0].lstrip("0")
        api.edit_song_arrangement(
            song_id=song["id"],
            arrangement_id=arrangement_id,
            source_id=song_source,
            source_ref=song_reference,
        )
    if iteration % 25 == 0:  # slow down api requests to avoid timeout
        TIMEOUT = 10
        logger.info("sleeping %s sec after %s/%s", TIMEOUT, iteration, len(songs))
        time.sleep(TIMEOUT)

# Mass import Gartenschau Termine

## prepare dataset

### from TXT

In [ ]:
import re
from pathlib import Path

import pandas as pd

pd.set_option("display.max_colwidth", None)

with Path("import/gartenschau.txt").open() as file:
    raw = file.readlines()

PATTERN = r"([\d.]+) +(\d\d:\d\d) +(.+)"


lines = []
for line in raw:
    line.replace("  ", " ")
    data = line[:-1].split(";")
    split = re.match(pattern=PATTERN, string=data[1].strip()).groups()
    lines.append([data[0].strip(), *split, *data[2:]])

df_imported = pd.DataFrame(lines)
df_imported

In [ ]:
df_imported["start_date"] = pd.to_datetime(
    df_imported[1] + "2025T" + df_imported[2], format="%d.%m.%YT%H:%M"
)
df_imported["title"] = df_imported[3]
df_imported["subtitle"] = df_imported[4]
df_imported["notes"] = df_imported[5]

### Cleanup using spreadsheet

In [ ]:
df_imported = df_imported[["start_date", "title", "subtitle", "notes"]]
df_imported["title"] = df_imported["title"].str.replace("Ökum. ", "")
df_imported = df_imported.fillna("")

Correcting with spreadsheed summary

In [ ]:
# df_imported.to_excel("import/gartenschau.xlsx")  # noqa: ERA001
df_new = pd.read_excel("import/gartenschau.xlsx")
df_new = df_new.fillna("")
df_imported["title"] = df_new.apply(
    lambda row: row["title"] + " (" + row["title2"] + ")"
    if row["title2"]
    else row["title"],
    axis=1,
)
disclaimer = " - Die Gottesdienste der Gartenschau werden von den Kirchen der ACK"
" Freudenstadt und Baiersbronn gemeinsam veranstaltet."
" Die Organisation des jeweiligen Termins obliegt der jeweiligen Gruppe,"
" kurzfristige Änderungen vorbehalten."
df_imported["notes"] = df_imported.apply(lambda row: row["notes"] + disclaimer, axis=1)
df_imported

## prepare common data for calendar appointment

In [ ]:
from datetime import timedelta

import pytz

address = {
    "meetingAt": "Volksbank Bühne",
    "city": "Baiersbronn",
    "street": "Am Königshammer",
    "country": "DE",
    "district": "Friedrichstal",
    "latitude": "48.4824546",
    "longitude": "8.3756814",
    "zip": "72270",
}

GARTENSCHAU_GODI = 73
GARTENSCHAU_KONZERT = 76
GARTENSCHAU_SONSTIGE = 82
GARTENSCHAU_KIDNER = 79

cest = pytz.timezone("Europe/Berlin")

create events

## Delete all existing Events

In [ ]:
from datetime import datetime

events = api.get_calendar_appointments(
    calendar_ids=[GARTENSCHAU_GODI],
    from_=(
        datetime(year=2025, month=5, day=1).astimezone(pytz.timezone("Europe/Berlin"))
    ),
    to_=(
        datetime(year=2025, month=10, day=30).astimezone(pytz.timezone("Europe/Berlin"))
    ),
)
for item in events:
    api.delete_calender_appointment(
        calendar_id=GARTENSCHAU_GODI, appointment_id=item["id"]
    )

## Create all new entries

In [ ]:
created_appointment_ids = []

for _index, line in df_imported.iterrows():
    new_id = api.create_calender_appointment(
        calendar_id=GARTENSCHAU_GODI,
        startDate=cest.localize(line["start_date"]),
        endDate=cest.localize(line["start_date"]) + timedelta(minutes=75),
        title=line["title"],
        subtitle=line["subtitle"],
        description=line["notes"],
        address=address,
    )
    created_appointment_ids.append(new_id)

created_ids = [item["id"] for item in created_appointment_ids]

# CCLI Tonality fix
## Prepare data
* Retrieve list of all songs
* filter to those that have a CCLI number and store relevant information for further processing

In [ ]:
songs = api.get_songs()

In [ ]:
import pandas as pd

EXPECTED_MIN_LENGTH_CCLI_ID = 4
relevant = [
    {
        "id": song["id"],
        "ccli": song["ccli"],
        "arrangement_ids": [arrangement["id"] for arrangement in song["arrangements"]],
    }
    for song in songs
    if len(song["ccli"]) > EXPECTED_MIN_LENGTH_CCLI_ID and song["ccli"] != "keine"
]

df_songs = pd.DataFrame(relevant)
df_songs

## Web Scraper CCLI
* Login into CCLI
* Request Song page
* Read and save tonality by CCLI ID

In [ ]:
from getpass import getpass
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

# set CCLI IDs requested based on previous step or own list
REQUESTED_CCLI_IDS = list(df_songs["ccli"])
# ["7096771","5588206","4904544"]  # noqa: ERA001

# Login credentials for CCLI SongSelect
USERNAME = input(prompt="Please enter your CCLI Username e.g. 'example@mymail.com' ")
PASSWORD = getpass("Please enter the corresponding password - will not be stored")

# Path to Firefox binary - sample is Ubuntu 24.04 compatible
firefox_binary_path = "/snap/firefox/5273/usr/lib/firefox/firefox"
options = Options()
options.binary_location = firefox_binary_path
driver = webdriver.Firefox(options=options, service=Service("/snap/bin/geckodriver"))

# Navigate to the login page
driver.get("https://profile.ccli.com/Account/Signin")
wait = WebDriverWait(driver, 20)  # Wait for up to 20 seconds
button = wait.until(
    expected_conditions.element_to_be_clickable(
        (By.ID, "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll")
    )
)
if button:
    button.click()

# Find the username and password fields and enter credentials
username_field = driver.find_element(By.NAME, "EmailAddress")
username_field.send_keys(USERNAME)
password_field = driver.find_element(By.NAME, "Password")
password_field.send_keys(PASSWORD)

del USERNAME, PASSWORD

# execute login
button = wait.until(expected_conditions.element_to_be_clickable((By.ID, "sign-in")))
if button:
    button.click()

ccli_tonality_map = {}

for ccli_id in REQUESTED_CCLI_IDS:
    sleep(1)
    driver.get(f"https://songselect.ccli.com/songs/{ccli_id}")
    wait.until(
        lambda driver: driver.execute_script("return document.readyState") == "complete"
    )
    sleep(4)
    result = next(
        item.text
        for item in driver.find_elements(by=By.CLASS_NAME, value="meta-data-group")
        if item.text.startswith("Standardtonart")
    )[15:]

    ccli_tonality_map[ccli_id] = result

# Close the driver
driver.quit()

# export based on PD dataframe
df_ccli = pd.DataFrame.from_dict(ccli_tonality_map, orient="index")
df_ccli.to_csv("import/ccli.csv")

## Merge CCLI mapping to existing song information
* import previously generated mapping
* rename columns
* merge dataframes

In [ ]:
df_ccli = pd.read_csv("import/ccli.csv", dtype=str)
df_ccli.columns = ["ccli", "tonart"]
df_songs_ccli = df_songs.merge(df_ccli, how="left", on="ccli")

In [ ]:
from time import sleep

for index, row in df_songs_ccli.iterrows():
    song_id = row["id"]
    arrangement_ids = row["arrangement_ids"]
    tonality = row["tonart"]
    logger.info(song_id, arrangement_ids, tonality)
    for a_id in arrangement_ids:
        api.edit_song_arrangement(
            song_id=song_id, arrangement_id=a_id, tonality=tonality
        )
    if index % 25 == 0:
        logger.info(
            "sleep 5 after 25 to avoid api overload @%s/%s", index, len(df_songs_ccli)
        )
        sleep(5)